In [17]:
# Seeting up project environment and API keys
from dotenv import load_dotenv
import os
import requests as rq
import json
import pandas as pd
import numpy as np
import warnings
import pytz
import datetime
import time
from IPython.display import clear_output
load_dotenv('.env') 
pd.set_option('display.precision', 4,
              'display.colheader_justify', 'center')
PUB_URL=os.getenv("COINGECKO_API_URL")
use_demo = {
    "accept": "application/json",
    "x-cg-demo-api-key" : os.getenv("COINGECKO_API_KEY")
}

def convert_to_local_tz(old_ts, desired_timezone="Asia/Singapore"):
    """
        This method converts the date to a proper timezone
    """
    new_tz = pytz.timezone(desired_timezone)
    old_tz = pytz.timezone("UTC")
    
    format = "%Y-%m-%dT%H:%M:%S+00:00"
    datetime_obj = datetime.datetime.strptime(old_ts, format)
    
    localized_ts = old_tz.localize(datetime_obj)
    new_ts = localized_ts.astimezone(new_tz)
    
    return new_ts

def get_response(endpoint, headers, params, URL):
    """ 
    Method to fetch data from external APIs
    Parameters:
        - endpoint (str): The API endpoint to fetch data from.
        - headers (dict): The headers to include in the request.
        - params (dict): The parameters to include in the request.
        - URL (str): The base URL of the API.

    Returns:
        - dict: The JSON response from the API if the request is successful.
        - None: If the request fails, prints an error message.
    """
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")

In [18]:
# list all crypto exchanges

exchange_params = {
    "per_page": 250,
    "page": 1
}
exchange_list_response = get_response("/exchanges", use_demo, exchange_params, PUB_URL)
df_ex = pd.DataFrame(exchange_list_response)

In [19]:
# list exchanges by trading volume
df_ex_subset = df_ex[["id", "name", "country", "trade_volume_24h_btc"]]
df_ex_subset = df_ex_subset.sort_values(by = ["trade_volume_24h_btc"], ascending = False)
df_ex_subset.head()

,id,name,country,trade_volume_24h_btc
0,binance,Binance,Cayman Islands,421608.2809
12,crypto_com,Crypto.com Exchange,Malta,89284.5671
150,fameex,FameEX,Australia,80479.1204
1,gdax,Coinbase Exchange,United States,78336.2004
20,mxc,MEXC,Seychelles,74276.1760


In [20]:
# list echanges by country 

df_ex_subset[(df_ex_subset.country=="United States")]

,id,name,country,trade_volume_24h_btc
1,gdax,Coinbase Exchange,United States,78336.2004
56,tapbit,Tapbit,United States,32566.8108
4,kraken,Kraken,United States,23197.1044
17,gemini,Gemini,United States,2623.8169
92,fmcpay,FMCPAY,United States,1433.8958
104,itbit,itBit,United States,468.1505
7,binance_us,Binance US,United States,227.9378
193,trade_ogre,TradeOgre,United States,42.3150
141,metalx,Metal X,United States,36.4337
143,oku-trade-bob-network,Oku Trade (BOB Network),United States,11.6807


In [21]:
def get_trade_exchange(id, base_curr, target_curr):
    
    exchange_ticker_response = get_response(f"/exchanges/{id}/tickers",
                                            use_demo,
                                            {},
                                            PUB_URL)
    
    found_match = ""
    
    for ticker in exchange_ticker_response["tickers"]:
        if ticker["base"] == base_curr and ticker["target"] == target_curr:
            found_match = ticker
            break
            
    # if found_match == "":
    #     warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
    
    return found_match

In [22]:
test_data = get_trade_exchange("gdax","ETH","USD")
test_data

{'base': 'ETH',
 'target': 'USD',
 'market': {'name': 'Coinbase Exchange',
  'identifier': 'gdax',
  'has_trading_incentive': False},
 'last': 1888.12,
 'volume': 444629.62484094,
 'converted_last': {'btc': 0.02356397, 'eth': 1.001729, 'usd': 1888.12},
 'converted_volume': {'btc': 10477, 'eth': 445399, 'usd': 839514087},
 'trust_score': 'green',
 'bid_ask_spread_percentage': 0.019066,
 'timestamp': '2025-03-11T07:03:55+00:00',
 'last_traded_at': '2025-03-11T07:03:55+00:00',
 'last_fetch_at': '2025-03-11T07:03:55+00:00',
 'is_anomaly': False,
 'is_stale': False,
 'trade_url': 'https://www.coinbase.com/advanced-trade/spot/ETH-USD',
 'token_info_url': None,
 'coin_id': 'ethereum'}

In [23]:
# Get cypto ticker data for multiple exchanges

def get_trade_exchange_per_country(country,
                                   base_curr,
                                   target_curr):
    """
        The bid-ask spread percentage is the difference between the lowest price asked for an asset by a seller and the highest bid made by a potential buyer. A lower value of the spread indicates higher liquidity and trading volume for the given asset on the exchange. Conversely, higher spread usually indicates lower liquidity. This metric can therefore be used to judge whether a particular exchange should be considered for executing arbitrage trades or not
    """
    
    df_all = df_ex_subset[(df_ex_subset["country"] == country)]    
    
    exchanges_list = df_all["id"]
    ex_all = []    
       
    for exchange_id in exchanges_list:
        found_match = get_trade_exchange(exchange_id, base_curr, target_curr)
        if found_match == "":
            continue
        else:
            temp_dict = dict(
                             exchange = exchange_id,
                             last_price = found_match["last"],
                             last_vol   = found_match["volume"],
                             spread     = found_match["bid_ask_spread_percentage"],
                             trade_time = convert_to_local_tz(found_match["last_traded_at"],'Africa/Kampala')
                             )
            ex_all.append(temp_dict)
            
    return pd.DataFrame(ex_all)

In [24]:
df_ex_all= get_trade_exchange_per_country("United States","ETH","USD")

In [25]:
df_ex_all

,exchange,last_price,last_vol,spread,trade_time
0,gdax,1888.12,444629.6248,0.0191,2025-03-11 10:03:55+03:00
1,kraken,1894.63,48538.4997,0.0105,2025-03-11 10:08:59+03:00
2,gemini,1894.52,19712.9989,0.0179,2025-03-11 10:09:30+03:00
3,itbit,1895.05,4864.9987,0.0126,2025-03-11 10:09:58+03:00
4,binance_us,1887.29,76.8167,0.3140,2025-03-11 10:04:11+03:00
5,coinzoom,1888.55,10.1419,0.0582,2025-03-11 10:05:16+03:00


In [26]:
# Get bitcoin exchange rates for multiple currencies

def get_exchange_rate(base_curr):
    
    # This returns current BTC to base_curr exchange rate    
    exchange_rate_response = get_response(f"/exchange_rates",
                                          use_demo,
                                          {},
                                          PUB_URL)
    rate = ""
    try:
        rate = exchange_rate_response["rates"][base_curr.lower()]["value"]
    except KeyError as ke:
        print("Currency not found in the exchange rate API response:", ke)
        
    return rate  

In [27]:
get_exchange_rate("USD")

80422.447

In [28]:
def get_vol_exchange(id:str, days:int, base_curr:str):
    """
    Parameters:
        - id (str): ID of the exchange
        - days (number): The history data you want in days.
        - base_curr (string): The base currency
    """
    
    vol_params = {"days": days}
    
    exchange_vol_response = get_response(f"/exchanges/{id}/volume_chart",
                                         use_demo,
                                         vol_params,
                                         PUB_URL)
    
    time, volume = [], []
    
    # Get exchange rate when base_curr is not BTC
    ex_rate = 1.0
    if base_curr != "BTC":
        ex_rate = get_exchange_rate(base_curr)
        
        # Give a warning when exchange rate is not found
        if ex_rate == "":
            print(f"Unable to find exchange rate for {base_curr}, vol will be reported in BTC")
            ex_rate = 1.0
    
    for i in range(len(exchange_vol_response)):
        # Convert to seconds
        s = exchange_vol_response[i][0] / 1000
        time.append(datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d'))
        
        # Default unit for volume is BTC
        volume.append(float(exchange_vol_response[i][1]) * ex_rate)
                      
    df_vol = pd.DataFrame(list(zip(time, volume)), columns = ["date", "volume"])
    
    # Calculate SMA for a specific window
    df_vol["volume_SMA"] = df_vol["volume"].rolling(7).mean()
    
    return df_vol.sort_values(by = ["date"], ascending = False).reset_index(drop = True)

In [ ]:
get_vol_exchange(base_curr="BTC",id="kraken",days=30)

In [30]:
def highlight_max_min(x, color):
    
    return np.where((x == np.nanmax(x.to_numpy())) |
                    (x == np.nanmin(x.to_numpy())),
                    f"color: {color};",
                    None)

def display_agg_per_exchange(df_ex_all, base_curr):
    
    # Group data and calculate statistics per exchange    
    df_agg = (
        df_ex_all.groupby("exchange").agg
        (        
            trade_time_min = ("trade_time", 'min'),
            trade_time_latest = ("trade_time", 'max'),
            last_price_mean = ("last_price", 'mean'),
            last_vol_mean = ("last_vol", 'mean'),
            spread_mean = ("spread", 'mean'),
            num_trades = ("last_price", 'count')
        )
    )
    
    # Get time interval over which statistics have been calculated    
    df_agg["trade_time_duration"] = df_agg["trade_time_latest"] - df_agg["trade_time_min"]
    
    # Reset columns so that we can access exchanges below
    df_agg = df_agg.reset_index()
    
    # Calculate % of total volume for all exchanges
    last_vol_pert = []
    for i, row in df_agg.iterrows():
        try:
            df_vol = get_vol_exchange(row["exchange"], 30, base_curr)
            current_vol = df_vol["volume_SMA"][0]
            vol_pert = (row["last_vol_mean"] / current_vol) * 100
            last_vol_pert.append(vol_pert)
        except:
            last_vol_pert.append("")
            continue
            
    # Add % of total volume column
    df_agg["last_vol_pert"] = last_vol_pert
    
    # Remove redundant column
    df_agg = df_agg.drop(columns = ["trade_time_min"])
    
    # Round all float values
    # (seems to be overwritten by style below)
    df_agg = df_agg.round({"last_price_mean": 2,
                           "last_vol_mean": 2,
                           "spread_mean": 2
                          })
    
    display(df_agg.style.apply(highlight_max_min,
                               color = 'green',
                               subset = "last_price_mean")
           )
           
    return None

In [31]:
def run_bot(country:str,
            base_curr:str,
            target_curr:str):
    
    df_ex_all = get_trade_exchange_per_country(country, base_curr, target_curr)
    
    # Collect data every minute    
    while True:
        time.sleep(60)
        df_new = get_trade_exchange_per_country(country, base_curr, target_curr)
        
        # Merge to existing DataFrame
        df_ex_all = pd.concat([df_ex_all, df_new])
        
        # Remove duplicate rows based on all columns
        df_ex_all = df_ex_all.drop_duplicates()
        
        # Clear previous display once new one is available
        clear_output(wait = True)
        display_agg_per_exchange(df_ex_all, base_curr)        
        
    return None

In [32]:
run_bot("United States","ETH","USDT")

,exchange,trade_time_latest,last_price_mean,last_vol_mean,spread_mean,num_trades,trade_time_duration,last_vol_pert
0,binance_us,2025-03-11 10:08:05+03:00,1889.860000,756.390000,0.280000,1,0 days 00:00:00,11.541901
1,coinlist,2025-03-11 10:06:34+03:00,1894.430000,3.860000,0.130000,1,0 days 00:00:00,10.175457
2,coinzoom,2025-03-11 10:08:19+03:00,1893.350000,4.000000,0.070000,1,0 days 00:00:00,1.580379
3,fmcpay,2025-03-11 10:07:27+03:00,1890.100000,1077.890000,0.030000,1,0 days 00:00:00,2.035476
4,gdax,2025-03-11 10:11:05+03:00,1897.100000,34815.670000,0.010000,1,0 days 00:00:00,1.562687
5,gemini,2025-03-11 10:06:33+03:00,1909.080000,70.630000,0.050000,1,0 days 00:00:00,0.087844
6,kraken,2025-03-11 10:09:00+03:00,1886.680000,8712.460000,0.020000,1,0 days 00:00:00,1.281372
7,tapbit,2025-03-11 10:10:34+03:00,1894.740000,930438.320000,0.010000,1,0 days 00:00:00,110.291412


KeyboardInterrupt: 